In [5]:
import configparser
import mysql.connector
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from datetime import datetime
import os

config = configparser.ConfigParser()
config.read('/Users/monky02/Desktop/FP/Config/config.ini')  # Ajusta ruta si es diferente

mysql_config = {
    'user': config.get('mysql', 'user'),
    'password': config.get('mysql', 'password'),
    'host': config.get('mysql', 'host'),
    'database': config.get('mysql', 'database'),
    'port': config.getint('mysql', 'port')
}

conexion = mysql.connector.connect(**mysql_config)

# ... resto de tu código sigue igual ...

mysql_config = {
    'user': config.get('mysql', 'user'),
    'password': config.get('mysql', 'password'),
    'host': config.get('mysql', 'host'),
    'database': config.get('mysql', 'database'),
    'port': config.getint('mysql', 'port')
}

conexion = mysql.connector.connect(**mysql_config)

# Consulta SQL para datos desde 2020 hasta 2025
query = """
SELECT
  fecha,
  UIDE,
  nombre_docente,
  nombre_asignatura
FROM TablasAnalisisJoin
WHERE YEAR(fecha) BETWEEN 2020 AND 2025
  AND UIDE IS NOT NULL AND UIDE != ''
ORDER BY fecha, nombre_asignatura;
"""

# Leer datos
df = pd.read_sql(query, conexion)
conexion.close()

# Limpiar texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
    palabras = texto.split()
    stopwords = set([
        'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las',
        'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'es', 'lo',
        'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'fue', 'esta', 'sí',
        'porque', 'esta', 'son', 'entre', 'cuando', 'muy', 'sin', 'sobre',
        'también', 'me', 'hasta', 'hay', 'donde', 'han', 'quien', 'está'
    ])
    palabras_filtradas = [p for p in palabras if p not in stopwords]
    return ' '.join(palabras_filtradas)

# Preprocesar
df['UIDE'] = df['UIDE'].astype(str).str.strip()
df['UIDE_limpio'] = df['UIDE'].apply(limpiar_texto)
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df['mes_anio'] = df['fecha'].dt.to_period('M')

# Crear carpeta raíz
os.makedirs("nubes_palabras", exist_ok=True)

# Agrupar por año, asignatura y mes
for (anio, asignatura, periodo), grupo in df.groupby(['anio', 'nombre_asignatura', 'mes_anio']):
    texto = ' '.join(grupo['UIDE_limpio'])
    if not texto.strip():
        continue

    # Crear subcarpeta: año/asignatura
    carpeta_asignatura = f"nubes_palabras/{anio}/{asignatura.replace(' ', '_').replace('/', '_')}"
    os.makedirs(carpeta_asignatura, exist_ok=True)

    # Crear y guardar la nube
    nube = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(nube, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"{asignatura} - {periodo}")
    plt.tight_layout()
    ruta = f"{carpeta_asignatura}/nube_{periodo}.png"
    plt.savefig(ruta)
    plt.close()

print("✅ Nubes de palabras generadas por año, materia y mes.")


/var/folders/km/1771f0nj2f9fwy4p84204mbw0000gn/T/ipykernel_7062/2473809296.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


✅ Nubes de palabras generadas por año, materia y mes.
